# Aula 3 - Exercícios - Pandas, Fontes de dados e Agregação
### Consulta a internet (Google, Stackoverflow e etc) liberada!!

In [1]:
#Fazer o download dos datasets
# Se já tiver feito o download, não precisa fazer novamente.
# Se aparecer um erro dizendo que a pasta não está vazia é porque o download já foi feito
!git clone https://github.com/alexlopespereira/curso_ciencia_dados2020.git ../../curso_ciencia_dados2020

fatal: destination path '../../curso_ciencia_dados2020' already exists and is not an empty directory.


In [7]:
# Importe as bibliotecas pandas, numpy, glob e sqlalchemy
import pandas as pd
import numpy as np
import sqlalchemy as sqla


### 3.1 Carregue o arquivo do IDEB 2017 no formato de um DataFrame.  Crie uma função para calcular a média do IDEB de todos os municípios de um determinado Estado. 
#### Remova os registros cujo valor na coluna REDE sejam Municipal, Estadual ou Federal. Deixe somente os registros cujos valores na coluna REDE sejam Pública.


In [19]:
# Dica 1: Antes de calcular, substitua valores '-' por np.nan (NA) com o metodo replace.
# Dica 2: Use a função loc para fazer o filtro de registros.
# Dica 3: Dentre várias maneiras possíveis, uma maneira bastante concisa é usar o operador & (and) para
# congregar dois critérios de filtro (seleção) de registros.
def media_ideb(df, sigla_estado):
    """retorne o valor da media do IDEB contido na coluna col do Estado especificado por sigla_estado.
    """
    # YOUR CODE HERE
    try:
        df.replace('-', np.nan, inplace=True)
        return df[df['REDE'] == 'Pública'].loc[sigla_estado, 'IDEB12_17'].mean()
    
    except ValueError as e:
        raise NotImplementedError(e)


In [20]:
## Faça seus testes aqui
path_ideb='../../curso_ciencia_dados2020/data/originais/ideb/ideb_municipios2017.xlsx'
df_ideb = pd.read_excel('../../curso_ciencia_dados2020/data/originais/ideb/ideb_municipios2017.xlsx', skiprows=9, skipfooter=3, index_col=0)
df_ideb[df_ideb['REDE'] == 'Pública'].loc['DF', 'IDEB12_17']


3.4

In [21]:
# Validação para nota
assert media_ideb(df_ideb, 'DF') == 3.4
assert round(media_ideb(df_ideb, 'SP'),2) == 4.04
assert round(media_ideb(df_ideb, 'GO'),2) == 4.24

### 3.2 Escreva uma função para salvar um DataFrame num banco de dados SQLite  

In [22]:

# Dica: A resolução deste exercício é bem semelhante ao código apresentado na Aula

def save_to_sqlite(df, table_name, con_str):
    """salve o dataframe df num banco de dados sqlite especificado por db_path
    """
    # YOUR CODE HERE
    try:
        #criando a conexão com o banco de dados
        con = sqla.create_engine(con_str)
        
        return df.to_sql(table_name, con, if_exists='replace')
        
    except ValueError as e:
        raise NotImplementedError('Erro ao tentar salvar no banco de dados.')


In [23]:
## Faça seus testes aqui
table_name = None # Escolha o nome da tabela
path_idhm='../../curso_ciencia_dados2020/data/originais/idhm/idhm2013.xlsx'
con_str='sqlite:////home/jovyan/work/curso_ciencia_dados2020/data/processados/idhm/idhm2013.sqlite'
df_idhm = pd.read_excel(path_idhm, sheet_name="IDHM")

# Antes de salvar no banco de dados, defina o nome do índice como index. Conforme exemplo abaixo
df_idhm.index.names = ['index']



In [24]:
path_idhm='../../curso_ciencia_dados2020/data/originais/idhm/idhm2013.xlsx'
con_str='sqlite:///../../curso_ciencia_dados2020/data/processados/idhm/idhm2013.sqlite'
table_name='IDHM'
df_idhm = pd.read_excel(path_idhm, sheet_name="IDHM")

df_idhm.index.names = ['index']
save_to_sqlite(df_idhm, table_name, con_str) 

In [25]:
# Validação para nota
def load_table(table, con_str):
    con = sqla.create_engine(con_str)
    return pd.read_sql_query("SELECT * from {0}".format(table), con, index_col='index')
    
path_idhm='../../curso_ciencia_dados2020/data/originais/idhm/idhm2013.xlsx'
con_str='sqlite:////home/jovyan/work/curso_ciencia_dados2020/data/processados/idhm/idhm2013.sqlite'
table_name='IDHM'
df_idhm = pd.read_excel(path_idhm, sheet_name="IDHM")

df_idhm.index.names = ['index']
save_to_sqlite(df_idhm, table_name, con_str) # Função que deve ser implementada pelo aluno
idhm_from_db = load_table(table_name, con_str)

assert idhm_from_db.equals(df_idhm)

### 3.3 Escreva uma função executar uma query SQL de agregação num banco de dados SQLite que retorne a média dos valores de uma coluna VALORES agregados por uma coluna CATEGORIAS.
#### Retorne a coluna CATEGORIA na tabela de resultados para identificar os valores. Ordene os resultados pelo valor da média em ordem decrescente.
#### Exemplo: Calcule média do IDHM (coluna VALORES) agregado por Ano da pesquisa (CATEGORIAS).

In [26]:
# Dica: A resposta deste exercício é bem semelhante ao código mostrado na aula
def soma_com_agregacao(val, cat, table, con_str):
    """retorne um dataframe contendo a soma dos valores em val dentro das categorias cat da tabela table
    """
    # YOUR CODE HERE
    try:
        #criando a conexão com o banco de dados
        con = sqla.create_engine(con_str)
        df_sqlite_agg = pd.read_sql_query("SELECT {1}, AVG({0}) as Media FROM {2} \
                                            GROUP BY {1} ORDER BY media DESC".format(val, cat, table), con)
        return df_sqlite_agg
    except:
        raise NotImplementedError()


In [27]:
## Faça seus testes aqui

soma_com_agregacao('IDHM', 'ANO', 'IDHM', con_str)

,ANO,Media
0,2010,0.659157
1,2000,0.523481
2,1991,0.381372


In [32]:
# Validação para nota
con_str='sqlite:////home/jovyan/work/curso_ciencia_dados2020/data/processados/idhm/idhm2013.sqlite'
idhm_anos = pd.DataFrame({'ANO': {0: 2010, 1: 2000, 2: 1991}, 'Media': {0: 0.6591574123989226, 1: 0.5234808625336926, 2: 0.38137232704402557}})
assert soma_com_agregacao('IDHM', 'ANO', 'IDHM', con_str).equals(idhm_anos)

### 3.4 Escreva uma função para fazer o merge dos dataframes de população e de PIB de 2017. A função deve receber como parâmetro o caminho para cada um dos dataframes.

In [71]:
# Dica 1: A coluna que contem o código IBGE do município deve ter o mesmo formato (string) e a mesma
#         quantidade de caracteres nas duas tabelas.
# Dica 2: Converta o codigo do município para string ao carregar o dataframe de população utilizando
#         o mapeamento de tipos dtype={'cod_munic': str, 'cod_uf': str}. Assim o codigo do municipio não perde
#         os zeros a esquerda.
# Dica 3: Concatene o codigo da UF com o código do município usando o operador + e salve em outra coluna.
# Dica 4: Use o código apresentado na aula para carregar o dataframe do PIB de 2017, especifique o dtype da
#         coluna que contém o codigo do municipio.
# Dica 5: Renomeie as colunas apropriadamente.
# Dica 6: Depois que os dois dataframes tiverem o codigo ibge (com valores equivalentes
#         e mesmo tipo de dados) você pode fazer o merge.

def merge_pop_pib(path_pop, path_pib):
    """retorne um dataframe resultante do merge dos dados do pib e da população
    """
    # YOUR CODE HERE
    try:
        df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, index_col=0, dtype={'Unnamed: 1': str})
        df_pib.rename(columns={'Unnamed: 1' : 'cod_ibge7', 'Unnamed: 2' : 'municipio'}, inplace=True)        
        
        df_pop = pd.read_excel(path_pop, dtype={'cod_uf': str, 'cod_munic': str})
        df_pop['cod_ibge7'] = df_pop['cod_uf'] + df_pop['cod_munic']
        
        resultado = pd.merge(df_pop, df_pib, on='cod_ibge7')
        resultado['municipio_y'] = resultado['municipio_y'].apply(lambda x: x[:-5])
        return resultado
        
    except ValueError as e:        
        raise NotImplementedError(e)


In [72]:
## Faça seus testes aqui
path_pib = '../../curso_ciencia_dados2020/data/originais/pib/pib_municipios.xlsx'
path_pop='../../curso_ciencia_dados2020/data/originais/populacao/estimativa_dou_2017.xlsx'

df_pib_pop = merge_pop_pib(path_pop, path_pib)
df_pib_pop.head()

,Unnamed: 0,uf,cod_uf,cod_munic,municipio_x,pop2017,cod_ibge7,municipio_y,2007,2009,2011,2013,2015,2017
0,0,RO,11,00015,Alta Floresta D'Oeste,25437,1100015,Alta Floresta D'Oeste,191364,256986,280510,341325,421300,498864
1,1,RO,11,00023,Ariquemes,107345,1100023,Ariquemes,905203,1133095,1651885,1799853,2037799,2296074
2,2,RO,11,00031,Cabixi,6224,1100031,Cabixi,49166,69776,77217,96365,116565,138894
3,3,RO,11,00049,Cacoal,88507,1100049,Cacoal,814890,985479,1259024,1433254,1794478,2127523
4,4,RO,11,00056,Cerejeiras,17934,1100056,Cerejeiras,143270,190902,260142,353270,397736,448524


In [73]:
# Validação para nota
test_merge = {'pop2017': 25437, '2017': 498864}
df_pib_pop = merge_pop_pib(path_pop, path_pib)
assert df_pib_pop.loc[0,['pop2017', '2017']].to_dict() == test_merge 

### 3.5 Escreva uma função para calcular o PIB percapita de cada município do Brasil, considerando o dataframe do exercício anterior.

In [74]:
# Dica 1
def pib_percapita(pib_pop):
    """retorne retorne um dataframe contendo o codigo ibge de 7 caracteres,
       o nome do municipio e seu PIB percapita.
    """
    # YOUR CODE HERE
    try:
        pib_pop['pib_percapita'] = pib_pop['2017'] / pib_pop['pop2017']

        return pib_pop
        
    except ValueError as e:        
        raise NotImplementedError(e)


In [75]:
## Faça seus testes aqui

pib_percapita(df_pib_pop)

,Unnamed: 0,uf,cod_uf,cod_munic,municipio_x,pop2017,cod_ibge7,municipio_y,2007,2009,2011,2013,2015,2017,pib_percapita
0,0,RO,11,00015,Alta Floresta D'Oeste,25437,1100015,Alta Floresta D'Oeste,191364,256986,280510,341325,421300,498864,19.611747
1,1,RO,11,00023,Ariquemes,107345,1100023,Ariquemes,905203,1133095,1651885,1799853,2037799,2296074,21.389669
2,2,RO,11,00031,Cabixi,6224,1100031,Cabixi,49166,69776,77217,96365,116565,138894,22.315874
3,3,RO,11,00049,Cacoal,88507,1100049,Cacoal,814890,985479,1259024,1433254,1794478,2127523,24.037907
4,4,RO,11,00056,Cerejeiras,17934,1100056,Cerejeiras,143270,190902,260142,353270,397736,448524,25.009702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5565,GO,52,22005,Vianópolis,13675,5222005,Vianópolis,122694,164336,223144,291919,312348,395787,28.942377
5566,5566,GO,52,22054,Vicentinópolis,8397,5222054,Vicentinópolis,87881,133430,149230,259422,322298,350765,41.772657
5567,5567,GO,52,22203,Vila Boa,5731,5222203,Vila Boa,31897,73367,84103,103233,111229,137500,23.992322
5568,5568,GO,52,22302,Vila Propício,5690,5222302,Vila Propício,76868,107125,112821,142970,198935,182175,32.016696


In [76]:
# Validação para nota
pib_percapita_2017 = pib_percapita(df_pib_pop)
assert round(pib_percapita(df_pib_pop).loc[0,'pib_percapita'],2) == 19.61

### 3.6 Escreva uma função para retornar os registros dos 10 maiores PIBs percapita do brasil a partir do dataframe do exercício anterior. 
#### Escreva uma função genérica que receba como parâmetro um dataframe e o nome da coluna de ordenação


In [77]:
# Dica 1: Use o método sort_values para ordenar o dataframe. Para uma ordenação descendente use ascending=False.
# Dica 2: Use o atributo iloc para retornar todas as colunas dos 10 primeiros registros.
def top10(df, col):
    """retorne os top 10 registros do dataframe df ordenados por col
    """
    # YOUR CODE HERE
    try:
        df_topten = pd.DataFrame(df.sort_values(col, ascending=False).iloc[:10])
        
        # esse trecho foi necessario por conta do assert
        df_topten.rename(columns={'municipio_x' : 'municipio'}, inplace=True)
        list_col_assert = ['cod_ibge7', 'municipio', 'uf', 'pib_percapita']
        
        return df_topten[list_col_assert]
    except ValueError as e:
        raise NotImplementedError(e)


In [78]:
## Faça seus testes aqui
top10(pib_percapita_2017, 'pib_percapita')

,cod_ibge7,municipio,uf,pib_percapita
3677,3536505,Paulínia,SP,344.847169
5068,4322004,Triunfo,RS,311.211920
3575,3527306,Louveira,SP,300.639389
3155,3204302,Presidente Kennedy,ES,292.397121
2965,3161908,São Gonçalo do Rio Abaixo,MG,289.925462
5174,5007802,Selvíria,MS,271.094724
2183,2929206,São Francisco do Conde,BA,253.895574
2523,3125101,Extrema,MG,219.239081
293,1508357,Vitória do Xingu,PA,209.799918
3547,3524709,Jaguariúna,SP,209.320862


In [79]:
# Validação para nota
test_data = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8471692406755, 3547: 209.32086192900894}}

assert top10(pib_percapita_2017, 'pib_percapita').iloc[[0,9]].to_dict() == test_data 

### 3.7 Escreva uma função para contar quantos nomes de municípios são diferentes nas bases de dados do PIB e da população. 
#### A coluna do nome dos municípios no dataset de PIB contém também o nome da UF. Remova o nome da UF, por exemplo, removendo os 5 ultimos caracteres da string do nome do município, antes de efetuar a comparação.

In [80]:
def compare_df_cols(df, col1, col2):
    """retorne a quantidade de registros em que o valor do registro na coluna col1 difere do valor em col2
    """
    # YOUR CODE HERE
    try:
        return len( df[df[col1] != df[col2]] )
        
    except ValueError as e:
        raise NotImplementedError(e)


In [82]:
## Faça seus testes aqui
# df_pib_pop.head()
compare_df_cols(df_pib_pop, 'municipio_x', 'municipio_y')


16

In [83]:
# Validação para nota
path_pib = '../../curso_ciencia_dados2020/data/originais/pib/pib_municipios.xlsx'
path_pop='../../curso_ciencia_dados2020/data/originais/populacao/estimativa_dou_2017.xlsx'
df_pib_pop = merge_pop_pib(path_pop, path_pib)
assert compare_df_cols(df_pib_pop, 'municipio_x', 'municipio_y') == 16

### 3.8 Considere algumas linhas de uma base de dados de morbidade apresentadas a seguir. Perceba que há dados de vários meses de 2020 para cada Estado da Federação.
#### Escreva uma função para pivotar esta base de dados apresentando os meses do ano nas colunas, os municípios nas linhas e a quantidade de óbitos como valores das células. 

|       | Município               |   cod_ibge6 | mes   |   ano | mes_ano    | Óbitos   |
|------:|:------------------------|------------:|:------|------:|:-----------|:---------|
|     0 | Alta Floresta D'Oeste   |      110001 | Abr   |  2020 | 2020-04-01 | 8        |
|     1 | Alto Alegre dos Parecis |      110037 | Abr   |  2020 | 2020-04-01 | 1        |
|     2 | Alto Paraíso            |      110040 | Abr   |  2020 | 2020-04-01 | 2        |
| 33347 | Vila Boa                |      522220 | Jan   |  2020 | 2020-01-01 | -        |
| 33348 | Vila Propício           |      522230 | Jan   |  2020 | 2020-01-01 | -        |
| 33349 | Brasília                |      530010 | Jan   |  2020 | 2020-01-01 | 578      |

In [84]:
# Dica 1: Sempre dê preferência para fazer agregação usando códigos como identificador, 
#         use nomes em ultimo caso quando não houver códigos identificadores. 
# Dica 2: Se você usar o atributo 'mes_ano' o pandas já ordena de forma cronológica as colunas.
def pivot_morbidade(file_path):
    """retorne um dataframe pivotado
    """
    # YOUR CODE HERE
    try:
        df_morbidade = pd.read_csv(file_path, sep=';')

        return df_morbidade.pivot(index=['cod_ibge6', 'Município'], columns='mes_ano', values='Óbitos')
            
    except ValueError as e:
        raise NotImplementedError(e)

In [85]:
## Faça seus testes aqui
file_path = '../../curso_ciencia_dados2020/data/originais/morbidade/consolidado/morbidade_simplificado.csv'
pivot_morbidade(file_path).iloc[0]


mes_ano
2020-01-01    5
2020-02-01    2
2020-03-01    3
2020-04-01    8
2020-05-01    2
2020-06-01    2
Name: (110001, Alta Floresta D'Oeste), dtype: object

In [86]:
# Validação para nota
file_path = '../../curso_ciencia_dados2020/data/originais/morbidade/consolidado/morbidade_simplificado.csv'
test_data = {'2020-01-01': '5', '2020-02-01': '2', '2020-03-01': '3', '2020-04-01': '8', '2020-05-01': '2', 
             '2020-06-01': '2'}
assert pivot_morbidade(file_path).iloc[0].to_dict() == test_data
